In [1]:
import pandas as pd
import seaborn as sns
import numpy as np
iort matplotlib.pyplot as plt
%matplotlib inline

In [2]:
#load the training data
train_values = pd.read_csv('training_set_values.csv')

In [3]:
#load the training set labels
train_labels = pd.read_csv('training_set_labels.csv')

In [4]:
#combine training and the labels for it
combined = train_values.merge(train_labels, on='id')

In [5]:
#check it worked
combined.head(5)

,id,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude,wpt_name,num_private,...,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group,status_group
0,69572,6000.0,2011-03-14,Roman,1390,Roman,34.938093,-9.856322,none,0,...,soft,good,enough,enough,spring,spring,groundwater,communal standpipe,communal standpipe,functional
1,8776,0.0,2013-03-06,Grumeti,1399,GRUMETI,34.698766,-2.147466,Zahanati,0,...,soft,good,insufficient,insufficient,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe,functional
2,34310,25.0,2013-02-25,Lottery Club,686,World vision,37.460664,-3.821329,Kwa Mahundi,0,...,soft,good,enough,enough,dam,dam,surface,communal standpipe multiple,communal standpipe,functional
3,67743,0.0,2013-01-28,Unicef,263,UNICEF,38.486161,-11.155298,Zahanati Ya Nanyumbu,0,...,soft,good,dry,dry,machine dbh,borehole,groundwater,communal standpipe multiple,communal standpipe,non functional
4,19728,0.0,2011-07-13,Action In A,0,Artisan,31.130847,-1.825359,Shuleni,0,...,soft,good,seasonal,seasonal,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe,functional


In [6]:
#start to remove columns that don't help
#recorded_by has no unique values

In [7]:
#check it worked
len(list(combined))

41

In [8]:
combined['recorded_by'].unique()

array(['GeoData Consultants Ltd'], dtype=object)

In [9]:
combined.drop('recorded_by', axis=1, inplace=True)

In [11]:
#look at categorical predictors that show similar information
#start with extraction_type, extraction_type_group, and extraction_type_class

d = combined
d.groupby(['extraction_type', 'extraction_type_group', 'extraction_type_class']).size()

extraction_type            extraction_type_group  extraction_type_class
afridev                    afridev                handpump                  1770
cemo                       other motorpump        motorpump                   90
climax                     other motorpump        motorpump                   32
gravity                    gravity                gravity                  26780
india mark ii              india mark ii          handpump                  2400
india mark iii             india mark iii         handpump                    98
ksb                        submersible            submersible               1415
mono                       mono                   motorpump                 2865
nira/tanira                nira/tanira            handpump                  8154
other                      other                  other                     6430
other - mkulima/shinyanga  other handpump         handpump                     2
other - play pump          other hand

In [12]:
# Group low occuring counts together or into related groups
# ignore warnings
combined['extraction_type'][combined.extraction_type.isin(['india mark ii','india mark iii'])] = 'india mark'
combined['extraction_type_class'][combined.extraction_type_class.isin(['india mark ii','india mark iii'])] = 'india mark'
combined['extraction_type'][combined.extraction_type.isin(['cemo', 'climax'])] = 'motorpump'
combined['extraction_type_class'][combined.extraction_type_class.isin(['cemo', 'climax'])] = 'motorpump'
combined['extraction_type'][combined.extraction_type.isin(['other - mkulima/shinyanga','other - play pump', 'walimi'])] = 'handpump'
combined['extraction_type_class'][combined.extraction_type_class.isin(['other - mkulima/shinyanga','other - play pump', 'walimi'])] = 'handpump'
combined['extraction_type'][combined.extraction_type.isin(['other - swn 81','swn 80'])] = 'swn'
combined['extraction_type_class'][combined.extraction_type_class.isin(['other - swn 81','swn 80'])] = 'swn'


C:\Users\julia.dills\AppData\Local\Continuum\Anaconda3\lib\site-packages\ipykernel\__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
C:\Users\julia.dills\AppData\Local\Continuum\Anaconda3\lib\site-packages\ipykernel\__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()
C:\Users\julia.dills\AppData\Local\Continuum\Anaconda3\lib\site-packages\ipykernel\__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-

In [13]:
#check it worked
d = combined
d.groupby(['extraction_type', 'extraction_type_group', 'extraction_type_class']).size()                       

extraction_type    extraction_type_group  extraction_type_class
afridev            afridev                handpump                  1770
gravity            gravity                gravity                  26780
handpump           other handpump         handpump                   135
india mark         india mark ii          handpump                  2400
                   india mark iii         handpump                    98
ksb                submersible            submersible               1415
mono               mono                   motorpump                 2865
motorpump          other motorpump        motorpump                  122
nira/tanira        nira/tanira            handpump                  8154
other              other                  other                     6430
other - rope pump  rope pump              rope pump                  451
submersible        submersible            submersible               4764
swn                other handpump         handpump          

In [14]:
#remove extraction_type_group
combined.drop('extraction_type_group', axis=1, inplace=True)

In [16]:
d = combined
d.groupby(['management', 'management_group']).size()      

management        management_group
company           commercial            685
other             other                 844
other - school    other                  99
parastatal        parastatal           1768
private operator  commercial           1971
trust             commercial             78
unknown           unknown               561
vwc               user-group          40507
water authority   commercial            904
water board       user-group           2933
wua               user-group           2535
wug               user-group           6515
dtype: int64

In [17]:
#keep these both

In [18]:
d = combined
d.groupby(['scheme_management', 'scheme_name']).size() 

scheme_management  scheme_name                 
Company            BFFS                              1
                   Bagamoyo Wate                    60
                   Bagamoyo wate                   210
                   Borehole                          1
                   Borehole drilling project         1
                   Bulamba piped scheme              2
                   Burieni water supply              1
                   Doe water supply                  3
                   EKTM 2 water  supply              1
                   EKTM 2 water suply               25
                   EKTM 2 water supply               9
                   EKTM 3 water supply              51
                   Fufulamsuri water supply          1
                   GRUMETI                           6
                   HESAWA                            1
                   Hakwe water supply               10
                   Holili water supply              23
                 

In [19]:
#remove scheme_name
combined.drop('scheme_name', axis=1, inplace=True)

In [21]:
d = combined
d.groupby(['payment', 'payment_type']).size() 

payment                payment_type
never pay              never pay       25348
other                  other            1054
pay annually           annually         3642
pay monthly            monthly          8300
pay per bucket         per bucket       8985
pay when scheme fails  on failure       3914
unknown                unknown          8157
dtype: int64

In [22]:
# these look the same so we can drop payment
combined.drop('payment', axis=1, inplace=True)

In [24]:
d = combined
d.groupby(['water_quality', 'quality_group']).size() 

water_quality       quality_group
coloured            colored            490
fluoride            fluoride           200
fluoride abandoned  fluoride            17
milky               milky              804
salty               salty             4856
salty abandoned     salty              339
soft                good             50818
unknown             unknown           1876
dtype: int64

In [25]:
#these look the same as well 
#keep the one with more description
combined.drop('quality_group', axis=1, inplace=True)

In [27]:
d = combined
d.groupby(['quantity', 'quantity_group']).size() 

quantity      quantity_group
dry           dry                6246
enough        enough            33186
insufficient  insufficient      15129
seasonal      seasonal           4050
unknown       unknown             789
dtype: int64

In [28]:
#these look the same as well 
#keep quantity
combined.drop('quantity_group', axis=1, inplace=True)

In [30]:
d = combined
d.groupby(['source', 'source_type', 'source_class']).size() 

source                source_type           source_class
dam                   dam                   surface           656
hand dtw              borehole              groundwater       874
lake                  river/lake            surface           765
machine dbh           borehole              groundwater     11075
other                 other                 unknown           212
rainwater harvesting  rainwater harvesting  surface          2295
river                 river/lake            surface          9612
shallow well          shallow well          groundwater     16824
spring                spring                groundwater     17021
unknown               other                 unknown            66
dtype: int64

In [31]:
#remove source_type
combined.drop('source_type', axis=1, inplace=True)

In [33]:
#should we drop other and unknown??

In [34]:
d = combined
d.groupby(['waterpoint_type', 'waterpoint_type_group']).size() 

waterpoint_type              waterpoint_type_group
cattle trough                cattle trough              116
communal standpipe           communal standpipe       28522
communal standpipe multiple  communal standpipe        6103
dam                          dam                          7
hand pump                    hand pump                17488
improved spring              improved spring            784
other                        other                     6380
dtype: int64

In [35]:
#drop one
combined.drop('waterpoint_type_group', axis=1, inplace=True)

In [37]:
#keep going with geographic info

In [38]:
d = combined
d.groupby(['region', 'region_code', 'district_code']).size() 

region         region_code  district_code
Arusha         2            1                 189
                            2                1206
                            3                 109
                            5                 201
                            6                 310
                            7                1009
               24           30                326
Dar es Salaam  7            1                  93
                            2                 497
                            3                 215
Dodoma         1            0                  23
                            1                 888
                            3                 361
                            4                 347
                            5                 358
                            6                 224
Iringa         11           1                 728
                            2                 530
                            3                 650
        

In [39]:
d = combined
d.groupby(['region', 'district_code', 'latitude', 'longitude']).size() 

region  district_code  latitude   longitude
Arusha  1              -3.766608  36.421219    1
                       -3.761918  36.436922    1
                       -3.761799  36.309273    1
                       -3.761723  36.309234    1
                       -3.761542  36.437071    1
                       -3.761492  36.437088    1
                       -3.761356  36.419235    1
                       -3.759929  36.419623    1
                       -3.685069  36.052036    1
                       -3.677827  36.604515    1
                       -3.677659  36.604690    1
                       -3.672770  36.054653    1
                       -3.672369  36.077617    1
                       -3.671986  36.051395    1
                       -3.669887  36.009399    1
                       -3.664992  35.963455    1
                       -3.663249  35.980449    1
                       -3.658037  36.051271    1
                       -3.640906  36.014173    1
                       -3

In [40]:
# Compute average longitude and latitude in districts within region to subsitute for zero values found
# i might do this later - might not be valid/useful

In [41]:
d = combined
d.groupby(['region', 'district_code']).describe()
#df['preTestScore'].groupby(df['company']).describe()

amount_tsh  construction_year   gps_height  \
region district_code                                                       
Arusha 1             count    189.000000         189.000000   189.000000   
                     mean       3.439153        1968.095238  1155.328042   
                     std       12.687854         250.670529   241.189337   
                     min        0.000000           0.000000   814.000000   
                     25%        0.000000        2000.000000   976.000000   
                     50%        0.000000        2000.000000  1055.000000   
                     75%        0.000000        2000.000000  1304.000000   
                     max       50.000000        2010.000000  1991.000000   
       2             count   1206.000000        1206.000000  1206.000000   
                     mean     694.361526        1972.573798  1435.470149   
                     std     3451.079819         214.192687   305.058904   
                     min        0.000000           0.000000   594.000000   
                     25%        0.000000        1990.000000  1087.250000   
                     50%        0.000000        2000.000000  1493.000000   
                     75%       20.000000        2003.000000  1654.750000   
                     max    20000.000000        2012.000000  2173.000000   
       3             count    109.000000         109.000000   109.000000   
                     mean      10.688073        2000.825688  1316.422018   
                     std       21.251012           2.765006    70.815265   
                     min        0.000000        2000.000000  1196.000000   
                     25%        0.000000        2000.000000  1250.000000   
                     50%        0.000000        2000.000000  1325.000000   
                     75%       20.000000        2000.000000  1366.000000   
                     max      100.000000        2010.000000  1433.000000   
       5             count    201.000000         201.000000   201.000000   
                     mean       2.437811        1970.706468  1740.268657   
                     std       10.125580         243.517512   518.073458   
                     min        0.000000           0.000000   612.000000   
                     25%        0.000000        1996.000000  1278.000000   
                     50%        0.000000        2005.000000  1851.000000   
...                                  ...                ...          ...   
Tanga  5             std      179.832382         825.028255    39.677798   
                     min        0.000000           0.000000   -39.000000   
                     25%        0.000000        1966.000000   -15.000000   
                     50%        0.000000        1980.000000     8.000000   
                     75%       20.000000        2008.000000    53.000000   
                     max     1500.000000        2011.000000   142.000000   
       6             count    266.000000         266.000000   266.000000   
                     mean      28.345865        1921.477444   412.086466   
                     std       19.893317         380.701390   151.533109   
                     min        0.000000           0.000000    75.000000   
                     25%       10.000000        1980.000000   323.500000   
                     50%       30.000000        2003.000000   375.500000   
                     75%       50.000000        2007.000000   512.000000   
                     max      100.000000        2011.000000   751.000000   
       7             count    127.000000         127.000000   127.000000   
                     mean     216.299213        1979.590551     0.000000   
                     std      566.317544         177.508075     0.000000   
                     min        0.000000           0.000000     0.000000   
                     25%        0.000000        1987.000000     0.000000   
                     50%        0.000000        1994.000000     0.000000   
  

In [42]:
d = combined
#d.groupby(['region', 'district_code']).describe()
d.groupby(['district_code','latitude']).mean()

id  amount_tsh  gps_height  longitude  \
district_code latitude                                                 
0             -6.576480   13230.0         0.0         0.0  36.526483   
              -6.576130   20639.0         0.0         0.0  36.526390   
              -6.540024    4892.0         0.0         0.0  36.554161   
              -6.537607   65229.0         0.0         0.0  36.556195   
              -6.528279    2457.0         0.0         0.0  36.554454   
              -6.452734   42743.0         0.0         0.0  36.631821   
              -6.452332   61381.0         0.0         0.0  36.629565   
              -6.451078   66073.0         0.0         0.0  36.641140   
              -6.448997   53128.0         0.0         0.0  36.711461   
              -6.448534   11576.0         0.0         0.0  36.709400   
              -6.446915   16535.0         0.0         0.0  36.707664   
              -6.446351   14678.0         0.0         0.0  36.707064   
              -6.443480    8133.0         0.0         0.0  36.722461   
              -6.442074   68455.0         0.0         0.0  36.704013   
              -6.438650   50855.0         0.0         0.0  36.720556   
              -6.437573   28718.0         0.0         0.0  36.700683   
              -6.436342   51195.0         0.0         0.0  36.723809   
              -6.435453   41830.0         0.0         0.0  36.712508   
              -6.429516   30603.0         0.0         0.0  36.734350   
              -6.422646   24185.0         0.0         0.0  36.700402   
              -6.417726   22976.0         0.0         0.0  36.663800   
              -6.414765    8660.0         0.0         0.0  36.680731   
              -6.410970   12416.0         0.0         0.0  36.666734   
1             -11.649440  20163.0      3000.0       330.0  37.351138   
              -11.648378  42800.0      3000.0       327.0  37.351869   
              -11.586297  57360.0         0.0       439.0  37.082702   
              -11.521312  43108.0         0.0       426.0  37.242288   
              -11.514937  48924.0         0.0       327.0  37.369605   
              -11.513886  70927.0         0.0       336.0  37.366089   
              -11.513231  42387.0         0.0       337.0  37.368038   
...                           ...         ...         ...        ...   
63            -7.814986   37710.0         0.0       -16.0  39.814419   
              -7.813973    3581.0         0.0       -22.0  39.827327   
              -7.811877   44172.0         0.0       -30.0  39.806648   
              -7.800661   35708.0         0.0       -29.0  39.821885   
              -7.800482   30729.0         0.0       -32.0  39.821660   
              -7.787924   36170.0         0.0        -8.0  39.860017   
              -7.786971   59868.0         0.0        -2.0  39.860960   
              -7.783063    2136.0         0.0       -16.0  39.854320   
              -7.782557   11718.0         0.0       -27.0  39.848522   
              -7.775643   41536.0         0.0       -40.0  39.842278   
              -7.775372   17653.0         0.0       -31.0  39.842987   
              -7.766395   44655.0         0.0       -13.0  39.873861   
67            -7.976036   53830.0         0.0       -14.0  39.739573   
              -7.973064    6032.0        20.0       -32.0  39.737884   
              -7.970700   44954.0        20.0       -27.0  39.740866   
              -7.970234   19987.0         0.0       -25.0  39.740798   
              -7.969308   43054.0         0.0       -30.0  39.740280   
              -7.968567   62904.0        30.0       -24.0  39.741519   
80            -3.969176   38236.0         0.0         0.0  34.006561   
              -3.960595   18435.0         0.0         0.0  34.008816   
              -3.956308    2510.0         0.0         0.0  34.014346   
              -3.955814   16336.0         0.0         0.0  34.021617   
              -3.955051   52590.0         0.0         0.0  34.111873   

In [49]:
#use lga column to create new feature: rural, urban, other
def get_setting(x):
    x=str(x)
    if x.find('Rural') >= 0: return 'rural'
    if x.find('Urban') >= 0: return 'urban'
    return 'other'

In [53]:
combined['rural_urban'] = combined.lga.apply(get_setting)

In [55]:
combined['rural_urban'].value_counts()

other    48165
rural     9552
urban     1683
Name: rural_urban, dtype: int64

In [56]:
print(type(combined.date_recorded))
print(type(combined.construction_year))

<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>


In [57]:
#use date time function in pandas
combined['date_recorded'] = pd.to_datetime(combined['date_recorded'])
combined['construction_year'] = combined.construction_year.map(lambda x: np.nan if x == 0 else x)

In [58]:
combined['operation_year'] = combined.date_recorded.dt.year - combined.construction_year

In [59]:
#print(type(combined.date_recorded))
combined[['operation_year', 'date_recorded', 'construction_year']].head(10)

,operation_year,date_recorded,construction_year
0,12.0,2011-03-14,1999.0
1,3.0,2013-03-06,2010.0
2,4.0,2013-02-25,2009.0
3,27.0,2013-01-28,1986.0
4,NaN,2011-07-13,NaN
5,2.0,2011-03-13,2009.0
6,NaN,2012-10-01,NaN
7,NaN,2012-10-09,NaN
8,NaN,2012-11-03,NaN
9,NaN,2011-08-03,NaN


In [60]:
#create season variable 
#look at monthly counts for reference
combined.date_recorded.dt.month.value_counts().sort_index()

1      6354
2     12402
3     17936
4      3970
5       336
6       346
7      6928
8      3364
9       328
10     5466
11     1349
12      621
Name: date_recorded, dtype: int64

In [61]:
combined['month_recorded'] = combined.date_recorded.dt.month

In [62]:
#Jan and Feb short dry season
#long rains lasts during about March, April and May 
#long dry season lasts throughout June, July, August, September and October 
#During November and December there's another rainy season: the 'short rains'.

In [64]:
def get_season (x):
    if x <= 2: return 1 #short dry season 
    elif x <= 5: return 2 #long rains
    elif x <= 10: return 3 #long dry season
    elif x<= 12: return 4 # short rains
    return 
    

In [65]:
#apply function and create new column
combined['season']=combined.month_recorded.apply(get_season)

In [66]:
combined.season.value_counts()

2    22242
1    18756
3    16432
4     1970
Name: season, dtype: int64

In [ ]:
#look at missing vals
combined.isnull().sum()

In [67]:
#export cleaned df - have removed a few features and engineered a few
#will clean both test and train dfs this way
pd.DataFrame(combined).to_csv("cleaned.csv")